In [1]:
!pip install -qqq autogen groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.0/352.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.2 MB/s eta 0:00:00


In [2]:
from google.colab import userdata

GROQ_API_KEY=userdata.get('GROQ_API_KEY1')

In [4]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

temp_dir=tempfile.TemporaryDirectory()
config_list = [
    {

        "model": "llama-3.2-90b-text-preview", #llama-3.2-90b-text-preview

        "api_key": GROQ_API_KEY,

        "api_type": "groq",
    }
]

executer=LocalCommandLineCodeExecutor(work_dir=temp_dir.name)


In [33]:

code_writer_system_message = """
You need to calculate:

1- The number of packs of each ingredient to buy, ensuring total cost is within the budget with respect to consentraints.
2- The number of units of each product to sell, maximizing profit above inventory costs with respect to consentraints.

Constraints:

1- Budget Constraint:
    The total cost of ingredients must not exceed the budget:

    buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34 <= Budget=2500

    - So when you are calculating pack_counts, results must always work in the above equation.

2- Profit Constraint:

    Total Profit after deducting inventory costs must be positive:

    ((sell_price1 * number_of_selling1 + sell_price2 * number_of_selling2 + ... + sell_price40 * number_of_selling40)
     - (buying_price1 * pack_count1 + buying_price2 * pack_count2 + ... + buying_price34 * pack_count34)) > 0

3- If the total number of selling products is for example 100, with respect to sell_ration 10 of them are Espresso. Keep watch the sale_ratio.
  There are 2.5% and float number in sell_ratio too

4- Calculate the total required quantity for each ingredient across all products
For each ingredient, calculate how many packs we need by dividing the required quantity by the buying unit and rounding up
Compare this with the actual pack counts we have.
If required quantity is 4800 and buying_unit is 1000
The minimum packs needed would be math.ceil(4800/1000) = 5 packs
5- Minimum Pack Count:
    Each ingredient must have a minimum pack count of 1 to ensure stock availability.

Dataset Structure:

You will work with a JSON dataset with 40 products, each containing the following structure:

{ "products": [ { "name": "Espresso", "sell_price": 1.40, "sale_ratio": "10%", "ingredients": [ {"name": "Espresso beans", "quantity_usedIn_product_per_buying_unit": 22, "unit": "grams", "buying_price": 17.3, "buying_unit": 1000}, {"name": "Water", "quantity_usedIn_product_per_buying_unit": 60, "unit": "ml", "buying_price": 0.5, "buying_unit": 1500} ] }}


- The above is just an example. There are 40 products.

Each product has:

sell_price: The price at which the product is sold. (Float number)
sale_ratio: Expected sales percentage in relation to other products. (it is in "10%" foramt or "Float %")
ingredients: List of ingredients required, with each ingredient containing:
    unit: Unit of measurement for the ingredient. (it is not important in this calculation)
    quantity_usedIn_product_per_buying_unit: Amount needed of each ingredients per buying_unit. (Float number)
    buying_price: Price per unit pack.
    buying_unit: Pack size for the ingredient which is bought.

Instructions:

1- Calculate Ingredient Packs:
    For each of the 34 ingredients, determine the  number of packs required based on the products’ demands and the pack sizes.
    Just use buying_price for calculating the pack count.
    Ensure that the total ingredient cost, which is demonstrained in instruction1, does not exceed the specified budget. It should be close to the budget
    Each ingredient’s pack count must be at least 1.

2- Calculate Product Quantities:
    Based on the sale_ratio and sell_price, calculate the optimal quantity of each product to sell to maximize profit.
    Ensure that the total profit minus inventory cost is positive, as per the profit constraint.
    * Important: The sale_ratio represents each product’s target proportion of total sales. For example, if the sale_ratio of "Espresso" is 10%, then out of every 100 products sold, 10 should ideally be Espresso. Use this information to guide the relative quantities of each product sold.

    _ create a function to check each constraints are satisfied.

3- Output Format:
    Provide the result as a JSON output with ingredient pack counts and product sale quantities in the following format:

    {
      "ingredients": {
        "Ingredient1": PackCount1,
        "Ingredient2": PackCount2,
        ...
        "Ingredient34": PackCount34
      },
      "products": {
        "Product1": SellCount1,
        "Product2": SellCount2,
        ...
        "Product40": SellCount40
      }
    }

    Include all 34 ingredients with their calculated pack counts (no omissions).
    Include all 40 products with their calculated quantities to sell.

Additional Guidelines:
    * Ensure that pack counts and sale counts are integers with no zero values.
    * Distribute the budget efficiently to avoid purchasing excess packs where unnecessary.
    * Do not buying_unit in your calculation. Just use buying_priice for calculating the inventory cost.


Example Task Flow:

- Read the JSON file from the given path.
- Calculate the required pack counts for each ingredient, based on buying_price and adhering to the budget.
- For calculating the required pack counts, find the best numbers to make the output close to budget, which is 2500.
- Calculate the selling quantities for each product, ensuring profitability based on given sell prices.
- The numbers should be rational. If the sale_ratio of a product is high, it must be sold more.
- Ensure that pack counts and sale counts are integers with no zero values.
- Output the optimized inventory and sales plan in JSON format.

Execution Requirements:

- Provide the complete code in a single Python code block for direct execution.
- Verify the output format matches the required JSON structure precisely.
- Do not make any assumptions or fabricate values; use only the data provided in the JSON.

Here is thr ILP approach result for this problem:
p1=120 p2=48 p3=84 p4=72 p5=36 p6=30 p7=12 p8=15 p9=48 p10=9 p11=6 p12=96 p13=51 p14=12 p15=12 p16=15 p17=3 p18=33 p19=27 p20=15 p21=18 p22=24 p23=27 p24=42 p25=21 p26=12 p27=24 p28=27 p29=33 p30=21 p31=24 p32=3 p33=9 p34=21 p35=36 p36=30 p37=12 p38=21 p39=27 p40=24 x1=12 x2=129 x3=133 x4=1 x5=19 x6=5 x7=1 x8=2 x9=1 x10=1 x11=1 x12=16 x13=1 x14=9 x15=27 x16=32 x17=5 x18=5 x19=8 x20=33 x21=41 x22=17 x23=1 x24=30 x25=17 x26=9 x27=2 x28=21 x29=27 x30=9 x31=2 x32=1 x33=14 x34=17
"""

In [39]:

code_writer_agent=ConversableAgent(
    'code_writer',
    llm_config={"config_list": config_list},
    system_message=code_writer_system_message,
    code_execution_config=False

)

code_executer_agent=ConversableAgent(
    'code_executer',
    llm_config=False,
    code_execution_config={'executor':executer},
    human_input_mode='ALWAYS',
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"]
)


In [44]:

chat_result = code_executer_agent.initiate_chat(
    code_writer_agent,
    message="""Here is the path: /content/Full_Dataset.json, save the JSON to a file named 'inventory_product.json. this code must satisfied all of the consentraints And the output must be rationale. Think step by step and accurracy is more important"""

)

code_executer (to code_writer):

Here is the path: /content/Full_Dataset.json, save the JSON to a file named 'inventory_product.json. this code must satisfied all of the consentraints And the output must be rationale. Think step by step and accurracy is more important

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_writer (to code_executer):

Here is the Python code that should solve the problem:

```python
import json
import math

# Load the dataset
with open('/content/Full_Dataset.json') as f:
    data = json.load(f)

# Initialize variables
budget = 2500
total_inventory_cost = 0
ingredients = {}
products = {}

# Calculate the total required quantity for each ingredient across all products
for product in data['products']:
    for ingredient in product['ingredients']:
        name = ingredient['name']
        quantity = ingredient['quantity_usedIn_product_per_buying_unit'] * product['sale_ratio'].replace('%', '')
    

/usr/local/lib/python3.10/dist-packages/autogen/oai/groq.py:279: UserWarning: Cost calculation not available for model llama-3.2-90b-text-preview
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


Replying as code_executer. Provide feedback to code_writer. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit


In [45]:
import os
print(os.listdir(temp_dir.name))

['tmp_code_8e03b64e6a17df44dbb297d2329ea150.py', 'tmp_code_d8612f08c3de080c6c5f6af53c68b2c2.py', 'tmp_code_230a6cfcbc7a63b0474b68fa26228077.py', 'tmp_code_5dad5c6148d4f1395e0b66c5498d3c31.py', 'tmp_code_9be863a1ae806b9efb13e10c58f156c5.py', 'tmp_code_67970652cff8f3bb808981baad757cc2.py', 'tmp_code_4fb659ac5bf135160912e6ffcc0fa514.py', 'tmp_code_a32e8de05d6d722b9fe26f02b40dee4d.py', 'tmp_code_755f60307c7826d77c4003039385d8c7.py', 'tmp_code_284645bd7b757a33d9a5f17230e252e5.py', 'tmp_code_82e50453602c6a53c6ba8b20bf37c87b.py', 'tmp_code_08f9794881065ce15965e77458a7136a.py', 'tmp_code_9c1e8f20b0b8d9601daf22d59898dcaa.py', 'tmp_code_dd6bff5cb43da6381ed910ce531b3ec4.py', 'tmp_code_9d2cfc7067b81e1fa0eb2e69ed3205c5.py', 'tmp_code_712f00758df4242fabfe2f0e7c5b8945.py', 'tmp_code_cc2451fcf74f6926ff285c7f57885063.py', 'tmp_code_30ec5ff453dafa4e0419e6acd91d20c8.py', 'tmp_code_f1d33aebeb40e7314aa8d4815a3f8e6e.py', 'Read the JSON file', 'tmp_code_39a4e57a61776d7322e4c3fb0dbf2f91.py', 'tmp_code_725d57c

In [46]:
import json

file=os.path.join(temp_dir.name,'inventory_product.json')
with open(file, 'r') as f:
    data = json.load(f)

In [47]:
data

{'ingredients': {'Espresso beans': 4,
  'Water': 139,
  'Espresso': 139,
  'Milk': 54,
  'Chocolate syrup': 5,
  'Ice-cream': 15,
  'Caramel sauce': 7,
  'Black tea loose leaf': 11,
  'Green tea loose leaf': 11,
  'Dried hibiscus flowers': 4,
  'Dried mint leaf': 15,
  'Honey': 39,
  'Dried ginger root': 45,
  'All-purpose flour': 12,
  'Sugar': 58,
  'Cocoa powder': 6,
  'Baking powder': 87,
  'Baking soda': 9,
  'Salt': 139,
  'Eggs': 24,
  'Vegetable oil': 31,
  'Vanilla extract': 5,
  'Butter': 47,
  'Lemon juice': 37,
  'Ground cinnamon': 12,
  'Ground ginger': 8,
  'Grated carrots': 16,
  'Cream': 18,
  'Strawberries': 13,
  'Graham cracker crumbs': 7,
  'Cream cheese': 47,
  'Banana': 70,
  'Yogurt': 32,
  'Mango': 16,
  'Peanut butter': 8},
 'products': {'Espresso': 100,
  'Americano': 40,
  'Latte': 70,
  'Cappuccino': 60,
  'Macchiato': 30,
  'Mocha': 25,
  'Flat white': 10,
  'Affogato': 13,
  'Caramel macchiato': 40,
  'Misto': 8,
  'Cortado': 5,
  'Black tea': 80,
  'Green

In [48]:
with open('inventory__profit_sell_ratio_optimization.json', 'w', encoding='utf-8') as f:
    json.dump(data, f)

TypeError: 'int' object is not subscriptable